In [ ]:
!pip3 install tensorflow
!pip3 install keras

In [2]:
# Tarek El-Hajjaoui
import numpy as np
import tensorflow as tf
import random
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [3]:
import string

def remove_punc(word, punc_set = string.punctuation):
  return ''.join(ch for ch in word if ch not in punc_set)

In [4]:
# get the text from the file
def gen_corpus_from_file(in_file="/content/nursery_rhymes.txt", 
                         out_file="/content/parsed_txt.txt"):
  headers_lst = []
  rhymes_lst = []
  skip_ln = lambda ln : True if (ln == "") else False
  is_header = lambda ln : True if (ln[-1].isupper()) else False
  with open(in_file) as file:
    for line in file:
      strip_line = line.replace('\r', ' ').replace('\n', ' ').strip()
      if skip_ln(strip_line): continue
      if is_header(strip_line):
        headers_lst.append(strip_line)
      else:
        rhyme = remove_punc(strip_line.lower())
        rhymes_lst.append(rhyme)
  rhymes = ' '.join(rhymes_lst)
  del headers_lst
  del rhymes_lst
  with open(out_file, "w") as out_f:
    out_f.write(rhymes)
  return rhymes

In [6]:
corpus = gen_corpus_from_file()

In [9]:
# LSTM Model
class LSTM_Text_Generator():
  def __init__(self, corpus, step_back, epochs):
    # member variables
    self.corpus = corpus
    self.vocab_size = None
    self.max_length = None
    self.step_back = step_back
    self.tokenizer = Tokenizer()
    self.encoded = None
    self.sequences = None
    self.X = None
    self.y = None
    self.tokenize_words()
    self.epochs = epochs
    self.next_seq = ''
    # Sequential is the base of the model
    self.model = Sequential()
  # integer encode sequences of words
  def tokenize_words(self):
    self.tokenizer.fit_on_texts([self.corpus])
    self.encoded = self.tokenizer.texts_to_sequences([self.corpus])[0]
    self.vocab_size = len(self.tokenizer.word_index) + 1
    print(f'Vocabulary Size: {self.vocab_size}')
  # encode step_back (2) words -> 1 word
  def gen_sequences(self):
    self.sequences = []
    for i in range(self.step_back, len(self.encoded)):
      sequence = self.encoded[i - self.step_back : i + 1]
      self.sequences.append(sequence)
    print(f'Total Sequences: {len(self.sequences)}')
    # pad sequences
    self.max_length = max([len(seq) for seq in self.sequences])
    self.sequences = pad_sequences(self.sequences, maxlen=self.max_length, padding='pre')
    print(f'Max Sequence Length: {self.max_length}')
  def dist_data_X_y(self):
    self.sequences = np.array(self.sequences)
    # split into input and output elements
    self.X, self.y = self.sequences[:,:-1], self.sequences[:,-1]
    self.y = to_categorical(self.y, num_classes=self.vocab_size)
  def pre_process(self):
    self.gen_sequences()
    self.dist_data_X_y()
  # compile network
  def compile(self):
    self.pre_process()
    self.model.add(Embedding(self.vocab_size, 10, input_length=self.max_length-1))
    self.model.add(LSTM(50))
    self.model.add(Dense(self.vocab_size, activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  def train(self):
    self.model.fit(self.X, self.y, epochs=self.epochs, verbose=2)
  def predict(self):
    return self.model.predict(self.encoded, verbose=0)
  def summary(self):
    return self.model.summary()
  def get_rand_word(self):
    rand_word = ''
    is_valid = False
    while is_valid == False:
      rand_word = self.corpus.split(' ')[random.randint(0, len(self.corpus.split(' ')) - 1)]
      if rand_word != ' ' and len(rand_word) != 0:
        is_valid = True
    return rand_word
  def reset_next_seq(self):
    self.next_seq = ''
  # generate a sequence from a language model
  def generate_seq(self, seed_seq='', supress_msg=True):
    if seed_seq == '':
      if supress_msg == False:
        print('No sequence was chosen, a random pair of words is being \
      chosen as the seed text.')
      seed_seq = get_rand_word() + " " + get_rand_word()
      if supress_msg == False:
        print(f"Random pair chosen: {seed_seq}")
    # when provide the model with a pair of words
    in_text = seed_seq
    # set the 2nd wor of current sequence as the first for the next sequence
    self.next_seq = seed_seq.split(' ')[1]
    # encode the text as integer
    encoded = self.tokenizer.texts_to_sequences([in_text])[0]
    # pre-pad sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=self.max_length-1, padding='pre')
    # predict probabilities for each word
    predict = self.model.predict(encoded, verbose=0)
    yhat=np.argmax(predict,axis=1)
    # map predicted word index to word
    out_word = ''
    for word, index in self.tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    # append to input
    in_text = out_word
    # set the out word as the 2nd word for next_seq variable
    self.next_seq += ' ' + out_word
    return in_text


In [10]:
def gen_nusery_rhymes(trained_model, n_lines=30,
                      n_words = 20, output_file="lstm_rhymes.txt"):
  with open(output_file, "w") as out_file:
    for i in range(n_lines):
      msg = f"Rhyme {i + 1}: "
      seed_pair = trained_model.get_rand_word() + " " + trained_model.get_rand_word()
      curr_line = trained_model.generate_seq(seed_pair)
      for n in range(n_words):
        curr_line += " " + trained_model.generate_seq(trained_model.next_seq)
      msg += curr_line
      out_file.write(msg + '\n')
      msg = ""
      curr_line = ""

In [11]:
LSTM_Model = LSTM_Text_Generator(corpus, 2, 400)

Vocabulary Size: 2393


In [12]:
LSTM_Model.compile()

Total Sequences: 15916
Max Sequence Length: 3


In [13]:
LSTM_Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 10)             23930     
                                                                 
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 dense (Dense)               (None, 2393)              122043    
                                                                 
Total params: 158,173
Trainable params: 158,173
Non-trainable params: 0
_________________________________________________________________


In [14]:
LSTM_Model.train()

Epoch 1/400
498/498 - 7s - loss: 6.7005 - accuracy: 0.0503 - 7s/epoch - 13ms/step
Epoch 2/400
498/498 - 2s - loss: 6.2089 - accuracy: 0.0525 - 2s/epoch - 4ms/step
Epoch 3/400
498/498 - 2s - loss: 6.0889 - accuracy: 0.0545 - 2s/epoch - 4ms/step
Epoch 4/400
498/498 - 2s - loss: 5.9924 - accuracy: 0.0621 - 2s/epoch - 4ms/step
Epoch 5/400
498/498 - 2s - loss: 5.9136 - accuracy: 0.0647 - 2s/epoch - 4ms/step
Epoch 6/400
498/498 - 2s - loss: 5.8396 - accuracy: 0.0683 - 2s/epoch - 4ms/step
Epoch 7/400
498/498 - 2s - loss: 5.7582 - accuracy: 0.0724 - 2s/epoch - 4ms/step
Epoch 8/400
498/498 - 2s - loss: 5.6727 - accuracy: 0.0752 - 2s/epoch - 4ms/step
Epoch 9/400
498/498 - 2s - loss: 5.5863 - accuracy: 0.0794 - 2s/epoch - 4ms/step
Epoch 10/400
498/498 - 2s - loss: 5.5008 - accuracy: 0.0832 - 2s/epoch - 4ms/step
Epoch 11/400
498/498 - 2s - loss: 5.4150 - accuracy: 0.0908 - 2s/epoch - 4ms/step
Epoch 12/400
498/498 - 2s - loss: 5.3269 - accuracy: 0.0953 - 2s/epoch - 4ms/step
Epoch 13/400
498/498 - 2

In [15]:
gen_nusery_rhymes(LSTM_Model, 30,
                  20, "lst_rhymes.txt")

In [16]:
LSTM_Model_2 = LSTM_Text_Generator(corpus, 2, 50)

Vocabulary Size: 2393


In [17]:
LSTM_Model_2.compile()

Total Sequences: 15916
Max Sequence Length: 3


In [18]:
LSTM_Model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2, 10)             23930     
                                                                 
 lstm_1 (LSTM)               (None, 50)                12200     
                                                                 
 dense_1 (Dense)             (None, 2393)              122043    
                                                                 
Total params: 158,173
Trainable params: 158,173
Non-trainable params: 0
_________________________________________________________________


In [19]:
LSTM_Model_2.train()

Epoch 1/50
498/498 - 4s - loss: 6.7023 - accuracy: 0.0513 - 4s/epoch - 7ms/step
Epoch 2/50
498/498 - 2s - loss: 6.2352 - accuracy: 0.0517 - 2s/epoch - 4ms/step
Epoch 3/50
498/498 - 2s - loss: 6.1415 - accuracy: 0.0525 - 2s/epoch - 4ms/step
Epoch 4/50
498/498 - 2s - loss: 6.0654 - accuracy: 0.0524 - 2s/epoch - 4ms/step
Epoch 5/50
498/498 - 2s - loss: 6.0110 - accuracy: 0.0528 - 2s/epoch - 4ms/step
Epoch 6/50
498/498 - 2s - loss: 5.9646 - accuracy: 0.0558 - 2s/epoch - 4ms/step
Epoch 7/50
498/498 - 2s - loss: 5.9072 - accuracy: 0.0648 - 2s/epoch - 4ms/step
Epoch 8/50
498/498 - 2s - loss: 5.8345 - accuracy: 0.0712 - 2s/epoch - 4ms/step
Epoch 9/50
498/498 - 2s - loss: 5.7496 - accuracy: 0.0749 - 2s/epoch - 4ms/step
Epoch 10/50
498/498 - 2s - loss: 5.6632 - accuracy: 0.0769 - 2s/epoch - 4ms/step
Epoch 11/50
498/498 - 2s - loss: 5.5824 - accuracy: 0.0794 - 2s/epoch - 4ms/step
Epoch 12/50
498/498 - 2s - loss: 5.5089 - accuracy: 0.0839 - 2s/epoch - 4ms/step
Epoch 13/50
498/498 - 2s - loss: 5.43

In [20]:
gen_nusery_rhymes(LSTM_Model_2, 30,
                  20, "lst_rhymes_2.txt")

Sources
- Jason Brownlee article: https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
- https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/